In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import r2_score
from skorch import NeuralNetRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge, LinearRegression

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# load training data
Xtr_loadpath = 'Xtr.csv'
Xts_loadpath = 'Xts.csv'
ytr_loadpath = 'ytr.csv'

Xtr = np.loadtxt(Xtr_loadpath, delimiter=",")
Xts = np.loadtxt(Xts_loadpath, delimiter=",")
ytr = np.loadtxt(ytr_loadpath, delimiter=",")

In [3]:
# standardize the training data
Xtr_mean = np.mean(Xtr,axis=0)
Xtr_std = np.std(Xtr,axis=0)
ytr_mean = np.mean(ytr)
ytr_std = np.std(ytr)

Xtr_standardized = ((Xtr-Xtr_mean[None,:])/Xtr_std[None,:]) # revise this line as needed
Xts_standardized = ((Xts-Xtr_mean[None,:])/Xtr_std[None,:]) # revise this line as needed
ytr_standardized = ((ytr-ytr_mean)/ytr_std)

# save the standardized training data
Xtr_savepath = 'Xtr_pytorch.csv'
Xts_savepath = 'Xts_pytorch.csv'
ytr_savepath = 'ytr_pytorch.csv'
yts_hat_savepath = 'yts_hat_pytorch.csv'

np.savetxt(Xtr_savepath, Xtr_standardized, delimiter=",")
np.savetxt(Xts_savepath, Xts_standardized, delimiter=",")
np.savetxt(ytr_savepath, ytr_standardized, delimiter=",")

In [4]:
lasso = Lasso()
lasso.set_params(alpha = 0.02, max_iter=10000, tol=0.01, warm_start=True)
lasso.fit(Xtr_standardized,ytr)
print(lasso.coef_)
feature_selected = abs(lasso.coef_).argsort()[-10:][::-1]
print(feature_selected)

[-1.2025827  -0.87100439 -0.69775652  2.91757736  0.         -1.68164859
  4.71752763 -0.53762284  1.65814527  0.36276424 -0.77379783  4.68714744
  0.87516852 -0.22619053  0.20134673  0.          0.         -0.38159997
  2.08580527 -3.0914095   1.73222023 -5.09691452  0.03116045  0.11419508
  0.02873892 -0.13480067]
[21  6 11 19  3 18 20  5  8  0]


In [5]:
print(Xtr_standardized.shape)
print(Xts_standardized.shape)
Xtr_featSel = Xtr_standardized[:,feature_selected]
Xts_featSel = Xts_standardized[:,feature_selected]
print(Xtr_featSel.shape)
print(Xts_featSel.shape)

(10000, 26)
(10000, 26)
(10000, 10)
(10000, 10)


In [6]:
# Convert the numpy arrays to PyTorch tensors
Xtr_torch = torch.Tensor(Xtr_featSel)
ytr_torch = torch.Tensor(ytr)

batch_size = 100  # size of each batch

# Create a training Dataset
train_ds = torch.utils.data.TensorDataset(Xtr_torch, ytr_torch)
# Creates a training DataLoader from this Dataset
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True) 


In [7]:
nin = Xtr_featSel.shape[1]
nout = 1

# model = nn.Sequential(
#     nn.Linear(nin, 128*5),
#     nn.ReLU(),
#     nn.Dropout(p=0.5),  # Add dropout layer with probability 0.2
#     nn.Linear(128*5, 64*5),
#     nn.ReLU(),
#     nn.Dropout(p=0.5),  # Add dropout layer with probability 0.2
#     nn.Linear(64*5, 32*5),
#     nn.ReLU(),
#     nn.Dropout(p=0.5),  # Add dropout layer with probability 0.2
#     nn.Linear(32*5, 16*5),
#     nn.ReLU(),
#     nn.Dropout(p=0.5),  # Add dropout layer with probability 0.2
#     nn.Linear(16*5, 8*5),
#     nn.ReLU(),
#     nn.Linear(8*5, 4*5),
#     nn.ReLU(),
#     nn.Linear(4*5, 2*5),
#     nn.ReLU(),
#     nn.Linear(2*5, nout)
# )

model = nn.Sequential(
    nn.Linear(nin, 2980),
    nn.ReLU(),
    nn.Dropout(p=0.5),  # Add dropout layer with probability 0.2
    nn.Linear(2980, 1096),
    nn.ReLU(),
    nn.Dropout(p=0.5),  # Add dropout layer with probability 0.2
    nn.Linear(1096, 403),
    nn.ReLU(),
    nn.Dropout(p=0.5),  # Add dropout layer with probability 0.2   
    nn.Linear(403, 148),
    nn.ReLU(),
    #nn.Dropout(p=0.5),  # Add dropout layer with probability 0.2  
    nn.Linear(148, 55),
    nn.ReLU(),
    #nn.Dropout(p=0.5),  # Add dropout layer with probability 0.2
    nn.Linear(55, 20),
    nn.ReLU(),
    nn.Linear(20, 7),
    nn.ReLU(),
    nn.Linear(7, 3),
    nn.ReLU(),
    nn.Linear(3, nout)
)

print(str(model))

Sequential(
  (0): Linear(in_features=10, out_features=2980, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=2980, out_features=1096, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=1096, out_features=403, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.5, inplace=False)
  (9): Linear(in_features=403, out_features=148, bias=True)
  (10): ReLU()
  (11): Linear(in_features=148, out_features=55, bias=True)
  (12): ReLU()
  (13): Linear(in_features=55, out_features=20, bias=True)
  (14): ReLU()
  (15): Linear(in_features=20, out_features=7, bias=True)
  (16): ReLU()
  (17): Linear(in_features=7, out_features=3, bias=True)
  (18): ReLU()
  (19): Linear(in_features=3, out_features=1, bias=True)
)


In [8]:
# Choosing the optimizer and loss function

epochs = 1
lrate = 1e-6
decay = lrate/epochs
lambda1 = lambda epoch: (1-decay)*epoch

opt = optim.Adam(model.parameters(), lr=lrate)
scheduler = optim.lr_scheduler.LambdaLR(opt, lr_lambda=lambda1)
criterion = nn.MSELoss()
#criterion = nn.HuberLoss(reduction='mean', delta=30)
#criterion = nn.L1Loss()

In [9]:
# training the model
num_epoch = epochs

a_tr_loss = np.zeros([num_epoch])
a_tr_Rsq = np.zeros([num_epoch])

for epoch in range(num_epoch):

    model.train() # put model in training mode
    batch_loss_tr = []
    batch_Rsq_tr = []
    # iterate over training set
    for train_iter, data in enumerate(train_loader):
        x_batch,y_batch = data
        print(x_batch.shape)
        y_batch = y_batch.view(-1,1)
        #y_batch = y_batch.type(torch.long)
        print(y_batch.shape)
        out = model(x_batch)
        # Compute Loss
        loss = criterion(out,y_batch.type(torch.float))
        batch_loss_tr.append(loss.item())
        # Compute R-square
        Rsq = r2_score(y_batch.type(torch.float).detach().numpy(), out.detach().numpy())
        batch_Rsq_tr.append(Rsq.item())
        # Compute gradients using back propagation
        opt.zero_grad()
        loss.backward()
        # Take an optimization 'step'
        opt.step()
        
    # Take scheduler step
    scheduler.step()
        
    a_tr_loss[epoch] = np.mean(batch_loss_tr) # Compute average loss over epoch
    a_tr_Rsq[epoch] = np.mean(batch_Rsq_tr)
    if (epoch+1) % 10 == 0:
        print('Epoch: {0:2d}   Train Loss: {1:.3f}   '.format(epoch+1, a_tr_loss[epoch])
             + 'R^2: {0:.3f}   '.format(a_tr_Rsq[epoch])
             )
    
with torch.no_grad():
    predict = model(torch.Tensor(Xtr_featSel)).detach().numpy().ravel()

r2 = r2_score(ytr,predict)
print('training R2: ',r2)
        

torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])
torch.Size([100, 10])
torch.Size([100, 1])


KeyboardInterrupt: 

In [ ]:
nepochs = len(a_tr_Rsq)
plt.plot(np.arange(1,nepochs+1), a_tr_Rsq, 'o-', linewidth=1)
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Train accuracy')

In [ ]:
nsteps = len(a_tr_loss)
ntr = Xtr.shape[0]
epochs = np.arange(1,nsteps+1)*batch_size/ntr
plt.semilogy(epochs, a_tr_loss)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.grid()
plt.xlim([0,np.max(epochs)])
plt.tight_layout()

In [ ]:
#Xts_standardized = Xts_standardized[:,feature_selected]
np.savetxt(Xtr_savepath, Xtr_featSel, delimiter=",")
np.savetxt(Xts_savepath, Xts_featSel, delimiter=",")

In [ ]:
# save the model: you must use the .pth format for pytorch models!
model_savepath = 'model.pth'

# To save a PyTorch model, we first pass an input through the model, 
# and then save the "trace". 
# For this purpose, we can use any input. 
# We will create a random input with the proper dimension.
x = torch.randn(10) # random input
x = x[None,:] # add singleton batch index
with torch.no_grad():
    traced_cell = torch.jit.trace(model, (x))

# Now we save the trace
torch.jit.save(traced_cell, model_savepath)

In [ ]:
# generate kaggle submission file using the validation script
!python {"validation.py " + model_savepath + " --Xts_path " + Xts_savepath + " --Xtr_path " + Xtr_savepath + " --yts_hat_path " + yts_hat_savepath } 